In [1]:
%load_ext autoreload
%autoreload 2
#from lib.multitasking_lib import *

In [2]:
######################
# Initiate Backtest  #
######################
from lib.multitasking_lib import *
cache_type ='backtest_uni'
my_cache = cache_state('backtest_uni')

def store_algo(algo, algo_name="default"):
    redis_conn.hset("algos",algo_name, algo)
    algo_f = open("log/"+algo_name+".txt", "w")
    algo_f.write(algo)
    algo_f.close()
    
#my_cache.flushall()
def backtest_unitest(stocks, freq='1T',algo_name = 'haikin_1_new', fromDate='2019-4-8', toDate='2019-4-9'):
    stocks = stocks
    sl = 1
    target = 1
    qty = 10
    backtest_msg={'stock':stocks,'sl':sl,'target':target,'qty':qty,'algo':algo_name,'fromDate':fromDate,'toDate':toDate,'freq':freq}

    # Step 4: Done is set to 0: Backtest is in progress, will be resetted by backtest job
    redis_conn.set('done'+cache_type,0)
    # Step 5: Send the msg to backtest thread to initiate the back test
    #pdebug(json.dumps(backtest_msg))
    redis_conn.publish('kite_simulator'+cache_type,json.dumps(backtest_msg))
    
##########################
# Unit Test              #
##########################

open('log/freedom.log','r+')
#log_file.close()

logger.setLevel(10)
loggerT.setLevel(21)
redis_conn.set('done'+cache_type,1)

try:
    if backtest_manager.job.is_alive():
        backtest_manager.job.terminate()
        time.sleep(0.1)
except:
    pass
finally:
    backtest_manager = threadManager(cache_type, ["kite_simulator","ohlc_tick_handler"], [kite_simulator, ohlc_tick_handler])
    
print(backtest_manager.job.is_alive())
print(redis_conn.pubsub_channels())
print(jobs)


INFO:	Cache Pointing to: backtest_uni
INFO:	Cache Pointing to: backtest_uni
INFO:	Starting Handler: kite_simulator


True
['tick_resamplerlive', 'freedom_initfreedom', 'ohlc_tick_handlerlive', 'kite_simulatorbacktest_web', 'order_handlerlive', 'ohlc_tick_handlerbacktest_web']
[<Process(Process-1, started)>]


INFO:	Starting Handler: ohlc_tick_handler
DEBUG:	kite_simulator: 1
ERROR:	kite_simulator: Invalid msg: 1
DEBUG:	kite_simulator: {"stock": ["TCS"], "sl": 1, "target": 1, "qty": 10, "algo": "haikin_1", "fromDate": "2019-4-8", "toDate": "2019-4-9", "freq": "1T"}
DEBUG:	Trade_init: TCS
INFO:	TCSbacktest_uni=>{'stock': 'TCS', 'qty': '0', 'SL %': '0.0', 'TP %': '0.0', 'amount': '0', 'price': '0.0', 'P&L': '0.0', 'P&L %': '0.0', 'Total P&L': '0.0', 'Total P&L %': '0.0', 'low': '0.0', 'sl': '0.0', 'ltp': '0.0', 'ltp %': '0.0', 'tp': '0.0', 'high': '0.0', 'last_processed': '1999-01-01', 'state': 'INIT', 'mode': 'PAUSE', 'algo': '', 'freq': '1D', 'hdf_freq': 'day'}
ERROR:	Error in executing algorithm
ERROR:	Error in executing algorithm
ERROR:	Error in executing algorithm
ERROR:	Error in executing algorithm
ERROR:	Error in executing algorithm
ERROR:	Error in executing algorithm
ERROR:	Error in executing algorithm
ERROR:	Error in executing algorithm
ERROR:	Error in executing algorithm
ERROR:	Error

In [3]:
backtest_unitest(['TCS'])

In [192]:
################################
###         Stop Trade       ###
################################

my_cache.delete('msgBufferQueue'+cache_postfix)
#my_cache.set('TCSbacktest_uniTICK',pd.DataFrame().to_json())

if backtest_manager.job.is_alive():
    backtest_manager.job.terminate()
time.sleep(1)
print(backtest_manager.job.is_alive())
print(redis_conn.pubsub_channels())

jobs
jobs = []

False
['tick_resamplerlive', 'freedom_initfreedom', 'ohlc_tick_handlerlive', 'kite_simulatorbacktest_web', 'kite_simulatorbacktest_uni', 'ohlc_tick_handlerbacktest_uni', 'order_handlerlive', 'ohlc_tick_handlerbacktest_web']


In [28]:
backtest_unitest(['TCS'])

ERROR:	Something went wrong while saving algo


In [ ]:
backtest_unitest(['WIPRO','TCS'])

In [27]:
from lib.charting_lib import *
cache_type = 'backtest_uni'
chart_type ='haikin'

symbol='TCS'
my_cache = cache_state(cache_type)

#######  Hack for freq  ############
my_cache.setValue('TCS','freq','1T')
#####################################
dfohlc = my_cache.getOHLC(symbol)
print(dfohlc.tail())
trade_df = pd.read_json(redis_conn.get(symbol+cache_type+'Trade'), orient='columns')

render_charts(dfohlc, trade_df, symbol, chart_type)

INFO:	Cache Pointer for: backtest_uni


                       close     high      low     open
2019-04-08 15:25:00  2074.95  2075.00  2074.65  2074.90
2019-04-08 15:26:00  2074.20  2074.95  2074.10  2074.95
2019-04-08 15:27:00  2073.40  2074.35  2073.40  2074.20
2019-04-08 15:28:00  2072.00  2073.50  2072.00  2073.50
2019-04-08 15:29:00  2073.60  2074.10  2071.20  2072.00


In [24]:
cache_state('backtest_uni').getValue()

DEBUG:	Cache initialized for: backtest_uni


,stock,qty,SL %,TP %,amount,price,P&L,P&L %,Total P&L,Total P&L %,...,ltp,ltp %,tp,high,last_processed,state,mode,algo,freq,hdf_freq
0,TCS,10,1,1,20661.0,0,49.00000000000091,2.377255967397677,294.4999999999959,1.4253908329703107,...,2073.6,0.0,0,2075.0,2019-04-08 15:29,SCANNING,PAUSE,haikin_1,1T,minute
1,WIPRO,10,1,1,2879.5,287.95,0,0,16.999999999999886,0.5903802743531823,...,288.1,0.0,290.8295,288.4,2019-04-23 12:24,LONG,PAUSE,haikin_1,1T,minute


In [183]:
###########################################################
###                  Quick Backtest                     ###
###########################################################
from talib import MACD, MACDEXT, RSI, BBANDS, MACD, AROON, STOCHF, ATR, OBV, ADOSC, MINUS_DI, PLUS_DI, ADX, EMA, SMA
from talib import LINEARREG, BETA, LINEARREG_INTERCEPT, LINEARREG_SLOPE, STDDEV, TSF, ADOSC, VAR, ROC, MIN, MAX, MINMAX

ohlc_get = lambda df, key: df.iloc[-1][key]
def HAIKINASI(ohlc_data_df):
    REF = lambda key, i: ohlc_get(ohlc_data_df.shift(i), key)
    
    OPEN  = ohlc_data_df['open']
    HIGH  = ohlc_data_df['high']
    LOW   = ohlc_data_df['low']
    CLOSE = ohlc_data_df['close']
    
    haOPEN  = (OPEN.shift(1) + CLOSE.shift(1))/2
    haHIGH  = pd.DataFrame([HIGH,OPEN,CLOSE]).max(axis = 0, skipna = True)
    haLOW   = pd.DataFrame([LOW,OPEN,CLOSE]).min(axis = 0, skipna = True)
    haCLOSE = (OPEN+HIGH+LOW+CLOSE)/4
    
    return (haOPEN, haHIGH, haLOW, haCLOSE)


################ Load Data ###################
from lib.charting_lib import *
cache_type = 'backtest_uni'
symbol='TCS'
my_cache = cache_state(cache_type)

###  Set Freq for Algo
my_cache.setValue('TCS','freq','1T')
dfohlc = my_cache.getOHLC(symbol)
#print(dfohlc.head())

### Load Data
ohlc_data_temp = dfohlc
pd.set_option('display.max_rows', None)
################ Initialize Helper Functions ###################
trade_df1 = pd.DataFrame()
OPEN = ohlc_data_temp['open']
CLOSE = ohlc_data_temp['close']
HIGH = ohlc_data_temp['high']
LOW = ohlc_data_temp['low']
#VOLUME = ohlc_data_temp['volume']

(haOPEN, haHIGH, haLOW, haCLOSE) = HAIKINASI(ohlc_data_temp)

TIME = ohlc_data_temp.index[-1].minute+ohlc_data_temp.index[-1].hour*60

#REF = lambda df, i: df.iloc[-i-1]
REF = lambda df, i: df.shift(i)
TREND_UP = lambda a,b: ROC(a, b) >= 0.1
TREND_DOWN = lambda a,b: ROC(a, b) <= -0.1
CROSSOVER = lambda a, b: (REF(a,1)<=REF(b,1)) & (a > b)


def BUY(x):
    global trade_df1
    tmp_df = pd.DataFrame()
    tmp_df["buy"] = CLOSE[x]
    trade_df1 = trade_df1.append(tmp_df)
    
def SELL(x):
    global trade_df1
    tmp_df = pd.DataFrame()
    tmp_df["sell"] = CLOSE[x]
    trade_df1 = trade_df1.append(tmp_df)
    
################# Write Algo For Backtest Here ###################

#sell = (REF(haOPEN, 0) > REF(haCLOSE,0)) & (REF(haOPEN, 1) > REF(haCLOSE,1))
#print(trade_df1)
sell = (ROC(CLOSE, 30) > 0.4) & (REF(haOPEN, 0) > REF(haCLOSE,0)) & (REF(haOPEN, 1) < REF(haCLOSE,1))
SELL( sell )
buy = (ROC(CLOSE, 30) < -0.4) & (REF(haOPEN, 0) < REF(haCLOSE,0)) & (REF(haOPEN, 1) > REF(haCLOSE,1))
BUY( buy )

logic = buy | sell

decision = 'WAIT'
if buy[-1] == True:
    decision = 'BUY'
elif sell[-1] == True:
    decision = 'SELL'
print(decision)

summary_df = trade_df1.copy(deep=True)
#summary_df['haopen'] = haOPEN[ logic ]
#summary_df['haclose'] = haCLOSE[ logic ]
summary_df['roc'] = ROC(CLOSE, 30)[logic]

#print(summary_df.sort_index())
print(summary_df.sort_index().tail(10))
#print(ROC(CLOSE,3))
#################### Render Charts ###############################
chart_type ='haikin'
#trade_df = pd.read_json(redis_conn.get(symbol+cache_type+'Trade'), orient='columns')

render_charts(ohlc_data_temp, trade_df1, symbol, chart_type)

INFO:	Cache Pointing to: backtest_uni


WAIT
                        sell      buy       roc
2019-04-08 10:59:00  2055.85      NaN  0.641292
2019-04-08 11:01:00  2055.80      NaN  0.712799
2019-04-08 11:05:00  2055.55      NaN  0.562609
2019-04-08 14:03:00  2066.85      NaN  0.681979
2019-04-08 14:06:00  2067.00      NaN  0.770281
2019-04-08 14:37:00      NaN  2057.05 -0.413923
2019-04-08 15:06:00  2066.00      NaN  0.510825
2019-04-08 15:18:00  2072.50      NaN  0.475105
2019-04-08 15:21:00  2072.80      NaN  0.606708
2019-04-08 15:26:00  2074.20      NaN  0.738222


In [188]:
cache_type = 'backtest_web'
symbol='TCS'
my_cache = cache_state(cache_type)
my_cache.getTrades(symbol)

my_cache.get('TCSbacktest_webTrades')

INFO:	Cache Pointing to: backtest_web


In [185]:
my_cache.getValue()

,stock,qty,SL %,TP %,amount,price,P&L,P&L %,Total P&L,Total P&L %,...,ltp,ltp %,tp,high,last_processed,state,mode,algo,freq,hdf_freq
0,TCS,10,1,1,0,0,0,0.0,0,0.0,...,2163.0,0.0,0.0,2165.5,2019-04-23 15:29,SCANNING,PAUSE,haikin_1_new,1T,minute
1,WIPRO,10,1,1,0,0,0,0.0,0,0.0,...,291.6,0.0,0.0,291.85,2019-04-23 15:29,SCANNING,PAUSE,haikin_1_new,1T,minute


In [ ]:
temp_file = pd.HDFStore("data/kite_cache.h5", mode="r")
temp_file.keys()

In [ ]:
# Long Strategies
def long_indicators(s):
    temp_df = 0
    #temp_df = temp_df | (REF(s.fastk,1) <=20)&(s.fastk>20)
    #temp_df = temp_df | CROSSOVER(s.fastk, s.fastd)
    #temp_df = temp_df | (REF(s.macd,1) <=0)&(s.macd>0)
    #temp_df = temp_df | CROSSOVER(s.macd, s.macdsignal)
    temp_df = temp_df | (REF(s.rsi,1) <=40)
    #temp_df = temp_df | (s.CLOSE.shift(1) == s.min.shift(1)) & (s.CLOSE > s.OPEN)
    #temp_df = temp_df & CROSSOVER(s.BBB, s.CLOSE)

    return pd.DataFrame(  temp_df, columns=["buy"] )

# Short Strategies
def short_indicators(s):
    temp_df = 0
    #temp_df = temp_df | (REF(s.fastk,1) >=80)&(s.fastk<80)
    #temp_df = temp_df | CROSSOVER(s.fastd, s.fastk)
    #temp_df = temp_df | (REF(s.macd,1) >=0)&(s.macd<0)
    #temp_df = temp_df | CROSSOVER(s.macdsignal, s.macd)
    temp_df = temp_df | (REF(s.rsi,1) >=60)
    #temp_df = temp_df & CROSSOVER(s.CLOSE, s.BBT)
    return pd.DataFrame( temp_df, columns=["sell"])



def long_breakout(s):
    temp_df = s.haCLOSE >= s.BBT.shift(1)
    temp_df = temp_df | (s.haCLOSE >= s.haOPEN.shift(2))
    temp_df = temp_df & ( CROSSOVER(s.OPEN, s.BBT) | CROSSOVER(s.OPEN, s.BBB) | CROSSOVER(s.OPEN, s.BBM))
    return pd.DataFrame( temp_df , columns=["buy"])

def short_breakout(s):
    temp_df = s.haOPEN <= s.BBB.shift(1)
    temp_df = temp_df | (s.haCLOSE <= s.haOPEN.shift(2))
    temp_df = temp_df & ( CROSSOVER(s.BBT, s.OPEN) | CROSSOVER(s.BBB, s.OPEN) | CROSSOVER(s.BBM, s.OPEN))
    return pd.DataFrame( temp_df , columns=["sell"])

def long_ha(s):
    temp_df = (REF(s.haCLOSE,3) < REF(s.haOPEN,3)) & (REF(s.haCLOSE,2) < REF(s.haOPEN,2)) & (REF(s.haCLOSE,1) > REF(s.haOPEN,1)) & (s.haCLOSE > s.haOPEN)  
    #temp_df = temp_df & (s.rsi < 40 )
    return pd.DataFrame( temp_df , columns=["buy"])

def short_ha(s):
    temp_df = (REF(s.haCLOSE,3) > REF(s.haOPEN,3)) & (REF(s.haCLOSE,2) > REF(s.haOPEN,2)) & (REF(s.haCLOSE,1) < REF(s.haOPEN,1)) & (s.haCLOSE < s.haOPEN)
    #temp_df = temp_df & (s.rsi > 60 )
    return pd.DataFrame( temp_df , columns=["sell"])
    


## Redis Console

In [ ]:
from redis import Redis
import json

conn = Redis(host='redis', port=6379, db=0, charset="utf-8", decode_responses=True)

In [ ]:
conn.xadd('msgBufferQueue',{'msg': 'next'})

In [ ]:
pd.set_option('display.max_rows', 50)
pd.read_json(conn.hget(stock+'_state','ohlc')).tail(50)

In [182]:
buy = pd.DataFrame()

if buy.shape[0] > 0 and buy[-1]==True:
    print('Yeah')

## Scratchpad

In [ ]:
redis_conn.pubsub_channels()

In [9]:
my_cache = cache_state('backtest_uni')

In [ ]:
df = my_cache.getValue()

In [13]:
pd.set_option('display.max_rows', 1000)
my_cache.getOHLC('TCS')

,close,high,low,open
2019-04-18 13:50:00,2144.00,2145.25,2144.00,2145.25
2019-04-18 13:51:00,2144.90,2145.00,2144.00,2144.00
2019-04-18 13:52:00,2143.75,2145.00,2143.75,2144.50
2019-04-18 13:53:00,2144.00,2144.00,2143.10,2143.75
2019-04-18 13:54:00,2144.70,2144.80,2143.80,2144.00
2019-04-18 13:55:00,2144.45,2144.80,2144.15,2144.70
2019-04-18 13:56:00,2144.80,2145.00,2144.45,2144.45
2019-04-18 13:57:00,2145.20,2145.50,2144.80,2144.80
2019-04-18 13:58:00,2145.95,2145.95,2145.05,2145.30
2019-04-18 13:59:00,2145.95,2145.95,2145.40,2145.40


In [ ]:
trades_df = my_cache.getTrades('TCS')
trades_df

In [ ]:
(total_profit, max_loss, max_profit, total_win, total_loss, max_winning_streak, max_loosing_streak, trade_log_df) = trade_analysis_raw(trades_df)

In [1]:
import pandas as pd
import plotly
import plotly.graph_objs as go
from datetime import datetime as dt
from datetime import timedelta
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from lib.multitasking_lib import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [8]:
# GetData

logger.setLevel(logging.DEBUG)
stock = 'WIPRO'
fromDate = '2019-4-22'
toDate  = '2019-4-24'
freq = 'minute'
mydate = fromDate
exchange = 'NSE'
ohlc_data = {}
ohlc_data[stock] = getData(stock, fromDate, toDate, exchange, freq, False, stock)

In [6]:
ohlc_data[stock].head()

,close,high,low,open,volume
date,,,,,
2019-04-22 09:15:00,285.90,286.50,285.15,285.50,167257
2019-04-22 09:16:00,286.45,286.55,285.60,285.85,128478
2019-04-22 09:17:00,286.40,286.55,286.00,286.25,105224
2019-04-22 09:18:00,286.80,287.00,286.35,286.40,96006
2019-04-22 09:19:00,287.65,287.80,286.80,286.80,100153


In [14]:
from lib.data_model_lib import *
live_cache = cache_state('backtest_web')
live_cache.getValue()
#live_cache.getOHLC('TCS', '1D').head()

DEBUG:	Cache initialized for: backtest_web


,stock,qty,SL %,TP %,amount,price,P&L,P&L %,Total P&L,Total P&L %,...,ltp,ltp %,tp,high,last_processed,state,mode,algo,freq,hdf_freq
0,WIPRO,10,1,1,2855.0,0,35.0,12.110726643598616,8.000000000000114,0.28021015761821766,...,298.55,0.0,0,299.45,2019-04-30 00:00,SCANNING,PAUSE,default,1D,day
1,TCS,10,1,1,22030.0,0,-450.0,-20.852641334569043,-1003.4999999999991,-4.555152065365407,...,2260.35,0.0,0,2266.95,2019-04-30 00:00,SCANNING,PAUSE,default,1D,day
